In [1]:
#!/usr/bin/env python3
import argparse
from importlib import import_module
from itertools import count
import os

import h5py
import json
import numpy as np
from sklearn.metrics import average_precision_score
import tensorflow as tf

import common
import loss

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.utils.np_utils import to_categorical
from keras.layers import LeakyReLU, Concatenate, concatenate, Lambda, UpSampling2D, Add, Input, Dense
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, BatchNormalization

In [2]:
class Args:
    excluder= 'duke' #, 'diagonal','cuhk03','duke'
    query_dataset = '/home/k/kajal/triplet-reid/data/duke/dukeMTMC_query.csv'
    query_embeddings = '/home/k/kajal/triplet-reid/experiments/duke_mobile/aug_query_embeddings.h5'
    #query_embeddings_adv = '/home/k/kajal/triplet-reid/experiments//MARKET/marketrecon255/test_embeddings.h5'
    gallery_dataset = '/home/k/kajal/triplet-reid/data/duke/dukeMTMC_test.csv'
    gallery_embeddings =  '/home/k/kajal/triplet-reid/experiments/duke_mobile/aug_gallery_embeddings.h5'
    metric =  'euclidean'
    #filename = 
    batch_size = 18

In [3]:
args=Args()

In [4]:
# Load the query and gallery data from the CSV files.
query_pids, query_fids = common.load_dataset(args.query_dataset, None)
gallery_pids, gallery_fids = common.load_dataset(args.gallery_dataset, None)

# Load the two datasets fully into memory.
with h5py.File(args.query_embeddings, 'r') as f_query:
    query_embs = np.array(f_query['emb'])
    query_embs_dp = np.array(f_query['emb'])
with h5py.File(args.gallery_embeddings, 'r') as f_gallery:
    gallery_embs = np.array(f_gallery['emb'])

In [5]:
gallery_embs=gallery_embs[228:1583]
gallery_pids=gallery_pids[228:1583]
query_embs=query_embs[4:231]
query_pids=query_pids[4:231]

#gallery_embs=gallery_embs[228:13316]
#gallery_pids=gallery_pids[228:13316]
#gallery_embs=gallery_embs[228:16654]
#gallery_pids=gallery_pids[228:16654]
#gallery_embs=gallery_embs[8095:]
#gallery_pids=gallery_pids[8095:]
#gallery_pids=gallery_pids[185:201],gallery_pids[228:16654]
uni_pids=np.unique(gallery_pids)
print(len(uni_pids))
uni_query_pids=np.unique(query_pids)
print(len(uni_query_pids))

66
66
66
66


In [6]:
uni_pids=np.unique(gallery_pids)
print(len(uni_pids))
uni_query_pids=np.unique(query_pids)
len(uni_query_pids)

66
66


66

66

In [7]:
print(query_embs.shape)
print(gallery_embs.shape)

(227, 128)
(1355, 128)
(227, 128)
(1355, 128)


In [8]:
#query_embs_dp=query_embs
query_embs_dp.shape
query_embs[0]
print(np.min(query_embs),np.mean(query_embs),np.max(query_embs),np.std(query_embs))

-5.0039964 -0.031202655 5.4245143 1.273047
-5.0039964 -0.031202655 5.4245143 1.273047


In [ ]:
query_embs_dp[0,1]=np.random.normal(0.021068161,2.312213)
query_embs_dp=np.exp(0.7) * query_embs_dp +0.001
print(query_embs[0])
print(query_embs_dp[0])

In [ ]:
def l1(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.max(np.subtract(actual,pred)).mean()

def mse(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.max(np.square(np.subtract(actual,pred)).mean())

D0 = query_embs + np.random.normal(0.021068161,2.312213)
D1 = query_embs_dp + np.random.normal(0.021068161,2.312213)
#print(D0,D1)
#sens1 = l1(query_embs,query_embs_dp)
sens2 = mse(D0,D1)
print(sens2)

In [9]:
query_data = query_pids
query_values = array(query_data)
print('values', query_values)
# integer encode
query_label_encoder = LabelEncoder()
query_integer_encoded = query_label_encoder.fit_transform(query_values)
print('encoded', query_integer_encoded)
# binary encode
query_onehot_encoder = OneHotEncoder(sparse=False)
query_integer_encoded = query_integer_encoded.reshape(len(query_integer_encoded), 1)
query_onehot_encoded = query_onehot_encoder.fit_transform(query_integer_encoded)
print(query_onehot_encoded.shape)

values ['0019' '0019' '0019' '0021' '0021' '0021' '0021' '0023' '0023' '0023'
 '0023' '0025' '0025' '0027' '0027' '0027' '0027' '0030' '0030' '0030'
 '0030' '0031' '0031' '0031' '0031' '0033' '0033' '0033' '0033' '0034'
 '0034' '0035' '0035' '0039' '0039' '0039' '0039' '0042' '0042' '0042'
 '0042' '0043' '0043' '0043' '0043' '0044' '0044' '0044' '0044' '0046'
 '0046' '0046' '0046' '0047' '0047' '0047' '0047' '0049' '0049' '0050'
 '0050' '0050' '0050' '0051' '0051' '0051' '0051' '0053' '0053' '0056'
 '0056' '0061' '0061' '0066' '0066' '0066' '0066' '0068' '0068' '0068'
 '0068' '0069' '0069' '0069' '0072' '0072' '0072' '0072' '0075' '0075'
 '0075' '0075' '0076' '0076' '0076' '0076' '0077' '0077' '0077' '0078'
 '0078' '0078' '0078' '0079' '0079' '0080' '0080' '0080' '0080' '0083'
 '0083' '0083' '0083' '0086' '0086' '0086' '0086' '0088' '0088' '0088'
 '0088' '0089' '0089' '0090' '0090' '0091' '0091' '0092' '0092' '0095'
 '0095' '0095' '0095' '0097' '0097' '0098' '0098' '0098' '0098' '0099'

In [10]:
data = gallery_pids
values = array(data)
print('values', values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print('encoded', integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded.shape)

values ['0019' '0019' '0019' ... '0140' '0140' '0140']
encoded [ 0  0  0 ... 65 65 65]
(1355, 66)
values ['0019' '0019' '0019' ... '0140' '0140' '0140']
encoded [ 0  0  0 ... 65 65 65]
(1355, 66)


In [11]:
X_train  = gallery_embs
Y_train_label = onehot_encoded
X_test = query_embs
Y_test_label = query_onehot_encoded
#print(X_train.size)
#print(X_test.size)
#print(Y_train_label)
#print(Y_test_label)

In [12]:
#print(Y_train_label[2:5000])
#y_train_label = to_categorical(Y_train_label)
#y_train_label.shape

In [13]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
#model.add( Dense(units=512, input_dim=128, kernel_initializer='normal', activation='relu') )
#model.add( Dropout(0.5))
model.add( Dense(units=66, input_dim=128, kernel_initializer='normal', activation='softmax') )
print( model.summary() )
model.compile( loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])

Model: "sequential"
Model: "sequential"
_________________________________________________________________
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 66)                8514      
 dense (Dense)               (None, 66)                8514      
                                                                 
                                                                 
Total params: 8,514
Total params: 8,514
Trainable params: 8,514
Non-trainable params: 0
Trainable params: 8,514
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
None
None


In [14]:
train_history = model.fit( x=gallery_embs, y=Y_train_label, validation_split=0.2, epochs=3, batch_size=128, verbose=2)

Epoch 1/3
Epoch 1/3
9/9 - 3s - loss: 4.1931 - accuracy: 0.0286 - val_loss: 4.2603 - val_accuracy: 0.0148 - 3s/epoch - 319ms/step
9/9 - 3s - loss: 4.1931 - accuracy: 0.0286 - val_loss: 4.2603 - val_accuracy: 0.0148 - 3s/epoch - 319ms/step
Epoch 2/3
Epoch 2/3
9/9 - 0s - loss: 3.4310 - accuracy: 0.2159 - val_loss: 4.4805 - val_accuracy: 0.0037 - 151ms/epoch - 17ms/step
9/9 - 0s - loss: 3.4310 - accuracy: 0.2159 - val_loss: 4.4805 - val_accuracy: 0.0037 - 151ms/epoch - 17ms/step
Epoch 3/3
Epoch 3/3
9/9 - 0s - loss: 2.7778 - accuracy: 0.4963 - val_loss: 4.7146 - val_accuracy: 0.0037 - 210ms/epoch - 23ms/step
9/9 - 0s - loss: 2.7778 - accuracy: 0.4963 - val_loss: 4.7146 - val_accuracy: 0.0037 - 210ms/epoch - 23ms/step


In [15]:
print("Evaluate on test data")
results = model.evaluate(X_test[4:2081], Y_test_label[4:2081])
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions)

Evaluate on test data
7/7 [==============================] - 0s 11ms/step - loss: 2.8761 - accuracy: 0.4709
test loss, test acc: [2.8760554790496826, 0.4708520174026489]
Generate predictions for 3 samples
test loss, test acc: [2.8760554790496826, 0.4708520174026489]
Generate predictions for 3 samples
8/8 [==============================] - 0s 1ms/step
predictions shape: [[0.06696355 0.00524317 0.00837805 ... 0.00411428 0.00324478 0.0008084 ]
 [0.06880886 0.00929948 0.02813125 ... 0.00433414 0.00664374 0.00257625]
 [0.04402748 0.0061026  0.0121971  ... 0.00657656 0.00696013 0.00242293]
 ...
 [0.01580173 0.05368307 0.01131012 ... 0.00559588 0.00394196 0.00647921]
 [0.01707824 0.03140106 0.00501237 ... 0.01451706 0.01465894 0.02496673]
 [0.0114368  0.03312156 0.02134123 ... 0.02341807 0.01441503 0.0241951 ]]
predictions shape: [[0.06696355 0.00524317 0.00837805 ... 0.00411428 0.00324478 0.0008084 ]
 [0.06880886 0.00929948 0.02813125 ... 0.00433414 0.00664374 0.00257625]
 [0.04402748 0.0061

In [ ]:
from diffprivlib.models import GaussianNB
clf = GaussianNB(epsilon=1,bounds=(-50e50, 50e50))
clf.fit(X_train, gallery_pids)

In [ ]:
clf.predict(X_test)
print("Test accuracy: %f" % clf.score(X_test, query_pids))

In [ ]:
def Hbeta(D=np.array([]), beta=1.0):
    """
        Compute the perplexity and the P-row for a specific value of the
        precision of a Gaussian distribution.
    """

    # Compute P-row and corresponding perplexity
    P = np.exp(-D.copy() * beta)
    sumP = sum(P)
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    P = P / sumP
    return H, P

def x2p(X=np.array([]), tol=1e-5, perplexity=30.0):
    """
        Performs a binary search to get P-values in such a way that each
        conditional Gaussian has the same perplexity.
    """

    # Initialize some variables
    print("Computing pairwise distances...")
    (n, d) = X.shape
    sum_X = np.sum(np.square(X), 1)
    D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
    P = np.zeros((n, n))
    beta = np.ones((n, 1))
    logU = np.log(perplexity)

    # Loop over all datapoints
    for i in range(n):

        # Print progress
        if i % 500 == 0:
            print("Computing P-values for point %d of %d..." % (i, n))

        # Compute the Gaussian kernel and entropy for the current precision
        betamin = -np.inf
        betamax = np.inf
        Di = D[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))]
        (H, thisP) = Hbeta(Di, beta[i])

        # Evaluate whether the perplexity is within tolerance
        Hdiff = H - logU
        tries = 0
        while np.abs(Hdiff) > tol and tries < 50:

            # If not, increase or decrease precision
            if Hdiff > 0:
                betamin = beta[i].copy()
                if betamax == np.inf or betamax == -np.inf:
                    beta[i] = beta[i] * 2.
                else:
                    beta[i] = (beta[i] + betamax) / 2.
            else:
                betamax = beta[i].copy()
                if betamin == np.inf or betamin == -np.inf:
                    beta[i] = beta[i] / 2.
                else:
                    beta[i] = (beta[i] + betamin) / 2.

            # Recompute the values
            (H, thisP) = Hbeta(Di, beta[i])
            Hdiff = H - logU
            tries += 1

        # Set the final row of P
        P[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))] = thisP

    # Return final P-matrix
    print("Mean value of sigma: %f" % np.mean(np.sqrt(1 / beta)))
    return P

def pca(X=np.array([]), no_dims=50):
    """
        Runs PCA on the NxD array X in order to reduce its dimensionality to
        no_dims dimensions.
    """

    print("Preprocessing the data using PCA...")
    (n, d) = X.shape
    X = X - np.tile(np.mean(X, 0), (n, 1))
    (l, M) = np.linalg.eig(np.dot(X.T, X))
    Y = np.dot(X, M[:, 0:no_dims])
    return Y
def tsne(X=np.array([]), no_dims=2, initial_dims=50, perplexity=30.0):
    """
        Runs t-SNE on the dataset in the NxD array X to reduce its
        dimensionality to no_dims dimensions. The syntaxis of the function is
        `Y = tsne.tsne(X, no_dims, perplexity), where X is an NxD NumPy array.
    """

    # Check inputs
    if isinstance(no_dims, float):
        print("Error: array X should have type float.")
        return -1
    if round(no_dims) != no_dims:
        print("Error: number of dimensions should be an integer.")
        return -1

    # Initialize variables
    X = pca(X, initial_dims).real
    (n, d) = X.shape
    max_iter = 10
    initial_momentum = 0.5
    final_momentum = 0.8
    eta = 200
    min_gain = 0.01
    Y = np.random.randn(n, no_dims)
    dY = np.zeros((n, no_dims))
    iY = np.zeros((n, no_dims))
    gains = np.ones((n, no_dims))

    # Compute P-values
    P = x2p(X, 1e-5, perplexity)
    P = P + np.transpose(P)
    P = P / np.sum(P)
    P = P * 4.									# early exaggeration
    P = np.maximum(P, 1e-12)
    

    # Run iterations
    for iter in range(max_iter):

        # Compute pairwise affinities
        sum_Y = np.sum(np.square(Y), 1)
        num = -2. * np.dot(Y, Y.T)
        num = 1. / (1. + np.add(np.add(num, sum_Y).T, sum_Y))
        num[range(n), range(n)] = 0.
        Q = num / np.sum(num)
        Q = np.maximum(Q, 1e-12)

        # Compute gradient
        PQ = P - Q
        for i in range(n):
            dY[i, :] = np.sum(np.tile(PQ[:, i] * num[:, i], (no_dims, 1)).T * (Y[i, :] - Y), 0)

        # Perform the update
        if iter < 20:
            momentum = initial_momentum
        else:
            momentum = final_momentum
        gains = (gains + 0.2) * ((dY > 0.) != (iY > 0.)) + \
                (gains * 0.8) * ((dY > 0.) == (iY > 0.))
        gains[gains < min_gain] = min_gain
        iY = momentum * iY - eta * (gains * dY)
        Y = Y + iY
        Y = Y - np.tile(np.mean(Y, 0), (n, 1))

        # Compute current value of cost function
        if (iter + 1) % 10 == 0:
            C = np.sum(P * np.log(P / Q))
            print("Iteration %d: error is %f" % (iter + 1, C))

        # Stop lying about P-values
        if iter == 100:
            P = P / 4.

    # Return solution
    return Y
             

In [ ]:
import pylab
X = gallery_embs[500:700]
lbl=list(map(int,gallery_pids))
labels = lbl[500:700]#query_pids[0:20]
pylab.figure()
Y = tsne(X, 2, 50, 20.0)
plt=pylab.scatter(Y[:, 0],Y[:,1],10,labels,marker='o')
pylab.axis('off')
pylab.savefig('rsltn3n1.pdf')
pylab.show()

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test_label)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions)

In [ ]:
import pylab
X = query_embs[500:700]
lbl=list(map(int,query_pids))
labels = lbl[500:700]#query_pids[0:20]
pylab.figure()
Y = tsne(X, 2, 50, 20.0)
plt=pylab.scatter(Y[:, 0],Y[:,1],10,labels,marker='o')
pylab.axis('off')
pylab.savefig('rsltn3n2.pdf')
pylab.show()

In [ ]:
    #file = query_embs
    #filel = query_pids
    X = query_embs[0:50]
    lbl=list(map(int,query_pids))
    labels = lbl[0:50]#query_pids[0:20]
    print(query_embs,query_pids)
    #X = np.loadtxt("mnist2500_X.txt")
    pylab.figure()
    Y = tsne(X, 2, 50, 20.0)
    plt=pylab.scatter(Y[:, 0],Y[:,1],30,labels,marker='+')
    pylab.axis('off')
    pylab.legend('on')
    pylab.savefig('rsltn3n1.pdf')
    pylab.show()

In [ ]:
    #file = query_embs
    #filel = query_pids
    X = query_embs[0:50]
    lbl=list(map(int,query_pids))
    labels = lbl[0:50]#query_pids[0:20]
    print(query_embs,query_pids)
    #X = np.loadtxt("mnist2500_X.txt")
    pylab.figure()
    Y = tsne(X, 2, 50, 20.0)
    plt=pylab.scatter(Y[:, 0],Y[:,1],30,labels,marker='+')
    pylab.axis('off')
    pylab.legend('on')
    pylab.savefig('rsltn3n1.pdf')
    pylab.show()